## Applying Rules Locally

we called out that Puppet is usually deployed in a client-server architecture. But that's not the only way we can use Puppet. **We can also use it as a stand-alone application run from the command line. This is common when testing new configurations.**

It can be the preferred configuration for complex setups where connecting to a master is no longer the best approach. When using a stand-alone Puppet, the same computer processes the facts, calculates the rules that need to be applied, and makes any necessary changes locally. 

**In Puppet lingo, these files are called manifests and they must end with a.pp extension.**

we'll just go with the Puppet packages provided by the Ubuntu distribution. We'll do that by installing the Puppet master package using sudo apt install puppet-master.

```
sudo apt install puppet-master
```

To do this, we first have to create a file where we'll store the rules that we want to apply. In Puppet lingo, these files are called manifests and they must end with a.pp extension. 

```
nano tool.pp
```

 We'll start by managing the htop package which is a tool similar to top that can show us some extra information. We'll state that we want Puppet to ensure that we have this package present on our computer.
 
```
sudo puppet apply -v tools.pp
```

The -v flag tells Puppet that we want to get verbose output which will tell us what's going on while Puppet is applying the rules in the file that we pass to it. 

Puppet first told us that it was loading the facts. Then, that it compiled a catalog. After that, it told us that it was applying the current configuration. Then, that it installed the package we requested. Finally, it let us know that it finished applying this catalog. 

You're probably wondering, what's a catalog? We called out in an earlier video that after loading all facts for a computer, the server calculates which rules actually need to be applied.

If a packet should only be installed when a certain condition is met, this condition is evaluated on the server side based on the gathered facts. **The catalog is the list of rules that are generated for one specific computer once the server has evaluated all variables, conditionals, and functions**. In this example, the catalog will be exactly the same as our code because the code didn't include any variables, functions, or conditionals.



## Managing Resource Relationships

The Puppet manifests that we use to manage computers in our fleet usually include a bunch of different resources that are related to each other. You're not going to configure a package that's not installed and you don't want to start a service until both the package and the configuration are in place. 

Puppets lets us control this with resource relationships. Let's check this out in an example. We have a file called ntp.pp, that has a bunch of resources related to the NUTS configuration.

This time, on top of declaring the resources that we need to manage, we're also declaring a few relationships between them.

```
class ntp{
    package { 'ntp':
        ensure => latest,
    }
    file{ '/etc/ntp.conf':
        source => '/home/user/ntp.conf',
        replace => true,
        require => Package['ntp'],
        notify => Service['ntp'],
     }
     service { 'ntp':
         enable => true,
         ensure => running,
         require => File['/etc/ntp.conf'],
      }
}

include ntp
```

**you might notice that the resource types are written in lowercase, but relationships like require or notify use uppercase for the first letter of the resource. This is part of Puppet syntax**. We write resource types in lowercase when declaring them, but capitalize them when referring to them from another resource's attributes. 

This time, on top of declaring the resources that we need to manage, we're also declaring a few relationships between them. We see that the configuration file requires the NTP package and the service requires the configuration file. 

This way, Puppet knows that before starting the service, the configuration file needs to be correctly set, and before sending the configuration file, the package needs to be installed. We're also declaring that the NTP service should be notified if the configuration file changes. That way, if we make additional changes to the contents of the configuration file in the future, the service will get reloaded with the new settings.

Now, one last thing. At the bottom of the file, we have a call to include NTP. That's why we told Puppet that we want to apply the rules described in a class. 